In [1]:
import pickle
import os,sys
import numpy as np
from baseline.AKE.ake import AKE
from baseline.Kriging.kriging import OrdryKriging
from baseline.NMF.nmf import NMF
from baseline.ST_MVL.st_mvl import ST_MVL

In [2]:
cd ..

/home/esmidth/github/CCS_TA_implement


In [3]:
with open('tensor2.pkl','rb') as f:
    tensor_array = pickle.load(f)

In [4]:
with open('new_indices.pkl','rb') as f:
    new_indices = pickle.load(f)

In [5]:
tensor_array = np.array(tensor_array)
tensor_array = tensor_array[new_indices]

In [6]:
def generate_missing_matrix(data_matrix,miss_ratio=0.1):
    origin  = data_matrix.copy()
    spatial_size = data_matrix.shape[0]
    temporal_size = data_matrix.shape[1]
    k = round(spatial_size*temporal_size*miss_ratio)
    mask = np.ones_like(data_matrix)
    ri = np.random.choice(spatial_size*temporal_size,k,replace=False)
    origin.flat[ri] = 0
    mask.flat[ri] = 0
    return origin,mask
data_matrix,mask =generate_missing_matrix(tensor_array[1])

In [7]:
ake = AKE()
rec = ake.predict(data_matrix)
print(np.sum(np.abs(rec-tensor_array[0])))

205.3832636678087


In [17]:
odKrigin = OrdryKriging()
odKrigin.train(data_matrix)
rec1 = odKrigin.predict()
print(np.sum(np.abs(rec1-tensor_array[0])))

170.75657462732306


In [8]:
nmf = NMF()
rec2 = nmf.predict(data_matrix,mask)
print(np.sum(np.abs(rec2-tensor_array[0])))

220.83192451207788


In [9]:
def get_distance_matrix(data_matrix,linear=False):
    if not linear:
        spatial_length = data_matrix.shape[0]
        # print(spatial_length)
        dis_matrix = np.random.rand(spatial_length,spatial_length)
        dis_matrix += dis_matrix.T
        np.fill_diagonal(dis_matrix,0)
        return dis_matrix
    else:
        spatial_length = data_matrix.shape[0]
        dis_matrix = np.zeros((spatial_length,spatial_length))
        for i in range(spatial_length):
            dis_matrix[i][i:] = np.arange(0,spatial_length-i)
        dis_matrix = dis_matrix + dis_matrix.T
        return dis_matrix

In [10]:
stmvl = ST_MVL()
ds = get_distance_matrix(data_matrix,True)
rec3 = stmvl.predict(data_matrix,ds)
print(np.sum(np.abs(rec3-tensor_array[1])))

7.395147142737379
